# Gaussian Target Practice

In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/Documents/PhD/MicroCanonicalHMC.jl/examples`
Precompiling project...
  ✓ RecursiveArrayTools → RecursiveArrayToolsForwardDiffExt
  ✓ ManifoldDiff → ManifoldDiffForwardDiffExt
NLSolversBase Waiting for background task / IO / timer.
[pid 55325] waiting for IO to finish:
 Handle type        uv_handle_t->data
 fs_event           0x399bda50->0x707b07c62710
 timer              0x397d7ec0->0x707b07c62740
This means that a package has started a background task or event source that has not finished running. For precompilation to complete successfully, the event source needs to be closed explicitly. See the developer documentation on fixing precompilation hangs for more help.
  ✓ NLSolversBase
  ✓ LineSearches
  ✓ Optim
  ✓ MicroCanonicalHMC
  ? NamedDims → TrackerExt
  ? RecursiveArrayTools → RecursiveArrayToolsTrackerExt
  ✓ Manifolds → ManifoldsRecipesBaseExt
  ✓ Symbolics → SymbolicsForwardDiffExt
  ✓ SymbolicAnalysis
  ✓ OptimizationBase
  ✓ OptimizationBase → Opt

In [2]:
# The statistical inference frame-work we will use
using Turing
using LinearAlgebra
using Random
#using StatsPlots
using PyPlot
using Distributed

# Extra deps
using LogDensityProblemsAD, LogDensityProblems

using Revise
using MicroCanonicalHMC

Precompiling Turing
  ✓ NamedDims → TrackerExt
  ✓ RecursiveArrayTools → RecursiveArrayToolsTrackerExt
  ✓ OptimizationBase → OptimizationTrackerExt
  ✓ Turing
  ✓ Turing → TuringOptimExt
  5 dependencies successfully precompiled in 17 seconds. 318 already precompiled.
[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
│   exception = Required dependency Rmath_jll [f50d1b31-88e8-58de-be2c-1cc44531875f] failed to load from a cache file.
└ @ Base loading.jl:1992
[ Info: Skipping precompilation since __precompile__(false). Importing Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0].
[ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
Precompiling DistributionsAD
    DistributionsDensityInterfaceExt Being precompiled by another process (pid: 52046, pidfile: /home/jaimerz/.julia/compiled/v1.10/DistributionsDensityInterf

In [ ]:
@model function model_Gaussian(rt)
    # Priors
    μ ~ Normal(0.3, 0.5)
    σ ~ Normal(log(0.2), 3)
    # Data
    rt ~ filldist(Normal(μ, exp(σ)), length(rt))
    end
end

fit = model_Gaussian(rand(100))

## MCHMC

In [ ]:
samples_MCHMC = sample(fit, externalsampler(MCHMC(200, 0.01; adaptive=true)), 200)

In [ ]:
theta_mchmc = [samples.value.data[i, 1, :][1] for i in axes(samples.value.data)[1]]
x10_mchmc = [samples.value.data[i, 10+1, :][1] for i in axes(samples.value.data)[1]];

## NUTS

In [ ]:
samples_NUTS = sample(fit, NUTS(), 200)  # Works

## Comp

In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(8,8))
fig.suptitle("Neal's Funnel Comp.", fontsize=16)

fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)

axis[1,1].hist(x10_mchmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="MCHMC")
axis[1,1].hist(x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS")
axis[1,1].hist(true_x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS - Truth")
axis[1,1].legend()
axis[1,1].set_yticks([])

axis[2,2].hist(theta_mchmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(true_theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6,2], alpha = 0.3)
axis[2,2].set_xticks([])
axis[2,2].set_yticks([])

axis[1,2].hist2d(true_x10_hmc, true_theta_hmc, bins=100, range=[[-6,2],[-4, 2]])
axis[1,2].set_xlabel("x10")
axis[1,2].set_ylabel("theta")
axis[1,2].set_title("NUTS")

axis[2,1].hist2d(x10_mchmc, theta_mchmc, bins=100, range=[[-6,2],[-4, 2]])
axis[2,1].set_xlabel("x10")
axis[2,1].set_ylabel("theta")
axis[2,1].set_title("MCHMC")